## Calcular Métricas - Tempo de Votação, Biometria, etc.

---

## Importing libraries

In [1]:
import duckdb
import pandas as pd
import time

## Importing Data

In [2]:
cursor = duckdb.connect()

Dados Brutos

In [3]:
TABLE = "read_parquet('VOTES.parquet/*/*/*/*.parquet', hive_partitioning=True)"
ZONE_GROUPS = [ (x, x+20) for x in range(0, 800, 20) ]

Adicionar TURNO e Timestamp final de Biometria

In [4]:
source = F"""
(
    SELECT 
        *,
        CASE event_date
            WHEN '2022-10-02' THEN 1
            WHEN '2022-10-03' THEN 1
            WHEN '2022-10-30' THEN 2
            WHEN '2022-10-31' THEN 2
            ELSE NULL
        END::INT AS turno,
        COALESCE(
            timestamp_biometria_manual,
            timestamp_biometria_4,
            timestamp_biometria_3,
            timestamp_biometria_2,
            timestamp_biometria_1
        ) AS timestamp_biometria_final
    FROM 
        {TABLE}
) _
"""

## Preparinga Data

Méticas no Cubo OLAP - Turno, UF, Zona, Seção.

- Número de Votos
- Número de Seções Eleitorais
- Média, Soma, q50%, q90% do Tempo total de Voto, Tempo de Biometria, Tempo Total

- Quantidade de Votos efetuados em até 30s, 1min, 1min30s, 2min, 2min30s, 3min+
- Taxa de Sucesso da Biometria em 1 tentativa, 2 tentativas, 3 tentativas, 4 tentativas, Falha
- Quantidade de Teclas Pressionadas
- Quantidade de Cargos Distintos Votados

**Definição das métricas de tempo**

In [5]:
tempo_voto_total = "EXTRACT(EPOCH FROM (timestamp_voto_computado  - timestamp_titulo_digitado))"
tempo_voto       = "EXTRACT(EPOCH FROM (timestamp_voto_computado  - timestamp_habilitacao_eleitor))"
tempo_biometria  = "EXTRACT(EPOCH FROM (timestamp_biometria_final - timestamp_titulo_digitado))"

intervalos_tempo_segundos_votos = [0, 30, 60, 90, 120, 150, 180, 210, 300, 9999]
contagem_de_votos_em_intervalos_de_tempo = ", ".join([
    F"""
    SUM( 
        CASE WHEN 
        {tempo_voto} >= {intervalos_tempo_segundos_votos[i]} 
        AND {tempo_voto} < {intervalos_tempo_segundos_votos[i+1]}
        THEN 1 ELSE 0 END 
    ) AS votos_{intervalos_tempo_segundos_votos[i]}_{intervalos_tempo_segundos_votos[i+1]}_segundos
    """
    for i in range(0, len(intervalos_tempo_segundos_votos)-1)
])

**Contagem de cargos distintos votados e número total de teclas pressionadas**

Aproximação a partir do número de digitos de cada cargo + 1 (CONFIRMA)

In [6]:
COLUNAS_VOTOS_CARGOS_NR_TECLAS = [
    # 2 digitos
    ('timestamp_voto_prefeito', 2), 
    ('timestamp_voto_presidente', 2),
    ('timestamp_voto_governador', 2),
    
    # 3 digitos
    ('timestamp_voto_senador', 3),

    # 4 digitos
    ('timestamp_voto_deputado_distrital', 4), 
    ('timestamp_voto_deputado_federal', 4),

    # 5 digitos
    ('timestamp_voto_deputado_estadual', 5),
]

nr_total_cargos_votados = " + ".join([
    F"({coluna} IS NOT NULL)::INT"
    for coluna, _ in COLUNAS_VOTOS_CARGOS_NR_TECLAS
])

nr_total_teclas_digitadas = " + ".join([
    F"({coluna} IS NOT NULL)::INT*({teclas}+1)"
    for coluna, teclas in COLUNAS_VOTOS_CARGOS_NR_TECLAS
])

In [7]:
fix_null_values = lambda column: F"COALESCE({column}::VARCHAR(10), 'ALL')"

In [8]:
query_metrics = F"""
    SELECT
        {fix_null_values('turno') } AS turno,
        {fix_null_values('uf') } AS uf,
        {fix_null_values('zone_code') } AS zone_code,
        {fix_null_values('section_code') } AS section_code,

        COUNT(*) AS total_votos,
        COUNT( DISTINCT uf || zone_code || section_code ) AS total_secoes,

        SUM( {tempo_voto} ) AS tempo_voto_soma,
        AVG( {tempo_voto} ) AS tempo_voto_medio,
        --PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY {tempo_voto}) AS tempo_voto_mediana,
        --PERCENTILE_CONT(0.9) WITHIN GROUP(ORDER BY {tempo_voto}) AS tempo_voto_90percentil,

        SUM( {tempo_biometria} ) AS tempo_biometria_soma,
        AVG( {tempo_biometria} ) AS tempo_biometria_medio,
        --PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY {tempo_biometria}) AS tempo_biometria_mediana,
        --PERCENTILE_CONT(0.9) WITHIN GROUP(ORDER BY {tempo_biometria}) AS tempo_biometria_90percentil,

        SUM( {tempo_voto_total} ) AS tempo_voto_total_soma,
        AVG( {tempo_voto_total} ) AS tempo_voto_total_medio,
        --PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY {tempo_voto_total}) AS tempo_voto_total_mediana,
        --PERCENTILE_CONT(0.9) WITHIN GROUP(ORDER BY {tempo_voto_total}) AS tempo_voto_total_90percentil,
        
        {contagem_de_votos_em_intervalos_de_tempo},
        1-AVG(biometria_nao_funcionou::INT) AS tx_sucesso_biometria,

        MAX({nr_total_cargos_votados})   AS nr_total_cargos_votados,
        SUM({nr_total_teclas_digitadas}) AS nr_total_teclas_digitadas

    FROM
        {source}
    WHERE quantidade_votos_computados = 1
    GROUP BY ROLLUP(turno, uf, zone_code, section_code)
"""

Os arquivos parquet são particionados por DATA DO EVENTO, UF e GRUPO DE ZONA ELEITORAL para agilizar a leitura dos dados pelo Dashboard.

As ZONAS foram agrupadas em grupos de 20, esse número é empírico.

In [9]:
query_metrics_with_zone_group = F"""
    SELECT
    *,
    CASE
        {
            "".join(
                [
                    f"WHEN zone_code!='ALL' AND zone_code::INT BETWEEN {min_zone} AND {max_zone} THEN '{min_zone}-{max_zone}' " 
                    for min_zone, max_zone in ZONE_GROUPS
                ]
            )
        }
        ELSE zone_code
    END AS zone_group
    FROM (
        {query_metrics}
    ) _
"""

In [10]:
query = F"""
    COPY (
    {
        query_metrics_with_zone_group
    } )
    TO 'VOTES_TIME_METRICS.parquet' 
    (FORMAT 'parquet', PARTITION_BY (turno, uf, zone_group), OVERWRITE_OR_IGNORE 1);
"""

In [11]:
cursor.execute(query)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [12]:
table = """
            read_parquet(
                'VOTES_TIME_METRICS.parquet/*/*/*/*.parquet', 
                hive_partitioning=True,
                hive_types_autocast=0
            )
        """
turno = 1
uf = 'DF'
zone_group = 'ALL'
zone = 1


query = f"""
            SELECT *
            FROM {table}
            WHERE 1=1
            AND turno = '{turno}'
            AND uf = '{uf}'
            -- AND zone_group = '{zone_group}'
            -- AND zone_code = {zone}
        """

df = cursor.execute(query).df()
df

,turno,uf,zone_code,section_code,total_votos,total_secoes,tempo_voto_soma,tempo_voto_medio,tempo_biometria_soma,tempo_biometria_medio,...,votos_90_120_segundos,votos_120_150_segundos,votos_150_180_segundos,votos_180_210_segundos,votos_210_300_segundos,votos_300_9999_segundos,tx_sucesso_biometria,nr_total_cargos_votados,nr_total_teclas_digitadas,zone_group
0,1,DF,0014,ALL,89318,308,4345747.0,48.654773,869184.0,10.492576,...,2633.0,906.0,409.0,237.0,276.0,174.0,0.953615,5,1745890.0,0-20
1,1,DF,0015,ALL,134744,505,6785043.0,50.355066,1171645.0,9.482934,...,4310.0,1623.0,837.0,425.0,494.0,244.0,0.963776,5,2667519.0,0-20
2,1,DF,0017,ALL,100240,378,5899118.0,58.849940,1344604.0,14.337855,...,5206.0,2062.0,1012.0,587.0,793.0,364.0,0.931993,5,1997143.0,0-20
3,1,DF,0005,0050,294,1,17810.0,60.578231,3067.0,11.193431,...,23.0,8.0,4.0,1.0,3.0,1.0,0.955782,5,5880.0,0-20
4,1,DF,0015,0229,281,1,14992.0,53.352313,2154.0,8.381323,...,16.0,5.0,1.0,0.0,1.0,0.0,0.975089,5,5620.0,0-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6625,1,DF,0021,0082,253,1,13693.0,54.122530,2232.0,9.073171,...,16.0,6.0,1.0,1.0,2.0,0.0,0.984190,5,5060.0,20-40
6626,1,DF,0021,0332,234,1,12897.0,55.115385,2474.0,10.850877,...,5.0,6.0,0.0,1.0,2.0,3.0,0.957265,5,4680.0,20-40
6627,1,DF,0021,0318,312,1,17853.0,57.221154,3879.0,13.104730,...,14.0,5.0,4.0,1.0,5.0,1.0,0.971154,5,6240.0,20-40
6628,1,DF,0021,0185,315,1,16997.0,53.958730,2877.0,9.558140,...,15.0,7.0,7.0,0.0,0.0,0.0,0.946032,5,6300.0,20-40


In [13]:
df.columns

Index(['turno', 'uf', 'zone_code', 'section_code', 'total_votos',
       'total_secoes', 'tempo_voto_soma', 'tempo_voto_medio',
       'tempo_biometria_soma', 'tempo_biometria_medio',
       'tempo_voto_total_soma', 'tempo_voto_total_medio',
       'votos_0_30_segundos', 'votos_30_60_segundos', 'votos_60_90_segundos',
       'votos_90_120_segundos', 'votos_120_150_segundos',
       'votos_150_180_segundos', 'votos_180_210_segundos',
       'votos_210_300_segundos', 'votos_300_9999_segundos',
       'tx_sucesso_biometria', 'nr_total_cargos_votados',
       'nr_total_teclas_digitadas', 'zone_group'],
      dtype='object')